# Bin Packing Lab

- Authors:
  - Jung Woo Lee, lee331@mcmaster.ca
  - Nicholas Fabugais-Inaba, fabugain@mcmaster.ca
- Group ID on Avenue: 56
- Gitlab URL: https://gitlab.cas.mcmaster.ca/lee331/l2-bin-packing

## How to use the provided code?

_(this section is just here for information, you can get rid of it in your own report)_

In [1]:
from macpacking.reader import DatasetReader, BinppReader
from macpacking.model  import Online, Offline
import macpacking.algorithms.offline as offline

Now that the business code is imported, we can load an existing dataset

In [2]:
dataset = '_datasets/binpp/N1C1W1/N1C1W1_B.BPP.txt'
reader: DatasetReader = BinppReader(dataset)
print(f'Dataset: {dataset}')
print(f'  - Bin Capacity: {reader.offline()[0]}')
print(f'  - Objects to pack: {sorted(reader.offline()[1])}')

Dataset: _datasets/binpp/N1C1W1/N1C1W1_B.BPP.txt
  - Bin Capacity: 100
  - Objects to pack: [8, 8, 12, 13, 13, 14, 15, 17, 18, 19, 20, 23, 30, 37, 37, 39, 40, 43, 43, 44, 44, 50, 51, 61, 61, 62, 62, 63, 66, 67, 69, 70, 71, 72, 75, 76, 76, 79, 83, 83, 88, 92, 92, 93, 93, 97, 97, 97, 99, 100]


Acording to the `oracle.xslx` file, we now that the optimal solution for this case is to use _31_ bins. Let's call the baseline algorithm, which is an offline one, and see how it performs.

In [3]:
import macpacking.algorithms.baseline as baseline
strategy: Offline = baseline.BenMaier()
result = strategy(reader.offline())
print(f'nb_bins = {len(result)}')
print(f'{sorted(result)}')

nb_bins = 31
[[40], [43, 43, 14], [50, 44], [51, 44], [61], [61, 39], [62], [62, 37], [63, 37], [66], [67], [69], [70, 30], [71], [72, 13, 13], [75, 18], [76, 19], [76, 23], [79, 20], [83, 15], [83, 17], [88, 12], [92, 8], [92, 8], [93], [93], [97], [97], [97], [99], [100]]


So the baseline finds the optimal solution. That's good news! Let's call our very own version of `NextFit`, as an offline algorithm.

In [3]:
import macpacking.algorithms.online as online
strategy: Offline = offline.NextFit()
result = strategy(reader.offline())
print(f'nb_bins = {len(result)}')
print(f'{sorted(result)}')

nb_bins = 35
[[14, 13, 13, 12, 8, 8], [20, 19, 18, 17, 15], [37, 30, 23], [39, 37], [43, 40], [44, 43], [50, 44], [51], [61], [61], [62], [62], [63], [66], [67], [69], [70], [71], [72], [75], [76], [76], [79], [83], [83], [88], [92], [92], [93], [93], [97], [97], [97], [99], [100]]


Damn it, this algorithm is 4 bins far from the optimal solution! Let's try an online version. Usually, they perform worst, so let's measure it.

In [4]:
# strategy: Online = online.NextFit()
# result = strategy(reader.online())
# print(f'nb_bins = {len(result)}')
# print(f'{sorted(result)}')

strategy: Online = online.FirstFit()
result = strategy(reader.online())
print(f'nb_bins = {len(result)}')
print(f'{result}')

nb_bins = 31
[[61, 13, 20], [62, 23, 15], [69, 8, 18], [83, 17], [97], [61, 37], [88, 8], [44, 43, 13], [72, 14, 12], [66, 30], [79, 19], [97], [83], [62, 37], [100], [70], [40, 44], [50, 43], [67], [63], [76], [97], [51, 39], [93], [93], [92], [76], [75], [71], [99], [92]]


As expected, the online version is worst!

## T1

INCOMPLETE, ADD DATASET ANALYSIS

Single responsibility 
Definition: Each class, function, or object is only responsible for one specific task. 

Enforcement: 
The code follows single responsibility as certain groups of tasks are delegated to their own classes and functions. Taking for example, the task of loading data is put into the reader classes in reader.py. Another example is how the business logic (i.e. the algorithms) are put into their own classes. The online and offline algorithms are further separated as their tasks are slightly different due to the nature of their inputs, and thus each algorithm has their own respective classes. There is no “super-function” that can complete all tasks through the use of conditional, the code is separated such that each section can handle their subset of inputs. Overall, by delegating specific tasks to specific classes, instead of all the tasks into a few large classes, the code follows single responsibility.


Open closed
Definition: The Open Closed principles states that an app or program is open for extension but closed for modification. 

Enforcement: 
The code follows this principle as it utilizes abstraction, which allows for easy future implementations. Taking for example, model.py, its classes Online and Offline are both abstract and they have abstract methods called _process(). _process() is left to be implemented by other classes, for its particular type of problem (online or offline bin-packing). If another offline algorithm is wished to be implemented, there is no change of existing code needed, a new class that implements _process() from the abstract Offline class is sufficient. The same applies for the abstract Online class, and DatasetReader; these abstract classes can be implemented by new classes in the future if wanted. Thus, the existing code need not be changed to add new functionality, making it open for extension and closed for modification.

Liskov’s Substitution Principle:
Definition:
Every subclass or derived class should be substitutable for their base or parent class.

Enforcement:
The code follows this principle as each of the abstract classes can be substituted for their subclasses. In model.py, the abstract classes both are expecting an output of type Solution for both of their _process() methods. In any call of the abstract classes, they can be substituted with one of their concretions. For example, if somewhere an Online object was created and _process() called for that object, we would be able to replace this with a NextFit_On object and its _process() call. There would be no resulting errors that occur. This applies for the Offline abstract class and its concretions, and the DatasetReader class with its concretion. 

Interface Segregation Principle
Definition: Whenever you have an interface, you need everything that implements that interface to all methods/parts of that interface (i.e., everything tracks back to one interface). 

Enforcement:
The code follows this principle as none of the concrete classes are required to implement unnecessary functions of what they are implementing. The NextFit classes only implement _process() from their abstract classes, and that is all that they require. BinppReader only implements _load_data_from_disk(), but does not need to implement online() or offline(), which would have been unnecessary for it to implement. 

Design for Interface 
Definition: Entities must depend on abstractions, not on concretions. It states that the high-level module must not depend on the low-level module, but they should depend on abstractions.

Enforcement: All classes in the code depend only on abstract classes, and no abstract class depends on a concrete class within the code, thus this principle is satisfied by the code.

## T2

PLOT BENCHMARK AND ANALYSIS

## T3

PLOT RESULTS AND ANALYSIS

## T4

PLOT RESULTS AND ANALYSIS

## T5

ANSWERS

## BONUS TASK

ANSWERS

## Self-reflection questions

As part of the self-reflection dimension of an experiential course, each member of the group is expected to answer to the following four questions:

  - What process did you go through to produce this result? (Backward)
  - What were your standards for this piece of work? Did you meet your standards? (Inward)
  - What the one thing you particularly want people to notice when they look at your work? (Outward)
  - What lessons will you keep from this reading/lecture in your professional practice? (Forward)